In [1]:
import cv2
import numpy as np
import os
import pytesseract as pt
import matplotlib.pyplot as plt

INPUT_WIDTH = 640
INPUT_HEIGHT = 640

img = cv2.imread('/home/deb/Imagens/merco/agile.jpeg')
net = cv2.dnn.readNetFromONNX('/home/deb/Downloads/YOLO1fev24/Model30-20240211T205252Z-001/Model30/weights/best.onnx')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

def get_detections(img, net):
    image = img.copy()
    row, col, d = image.shape
    max_rc = max(row, col)
    input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
    input_image[0:row, 0:col] = image

    blob = cv2.dnn.blobFromImage(input_image, 1 / 255, (INPUT_WIDTH, INPUT_HEIGHT), swapRB=True, crop=False)
    net.setInput(blob)
    preds = net.forward()
    detections = preds[0]
    return input_image, detections

def non_maximum_suppression(input_image, detections):
    boxes = []
    confidences = []

    image_w, image_h = input_image.shape[:2]
    xfactor = image_w / INPUT_WIDTH
    yfactor = image_h / INPUT_HEIGHT

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4]  
        if confidence > 0.4:
            class_score = row[5]
            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                left = int((cx - 0.5 * w) * xfactor)
                top = int((cy - 0.5 * h) * yfactor) 
                width = int(w * xfactor)
                height = int(h * yfactor)
                box = np.array([left, top, width, height])
                confidences.append(confidence)
                boxes.append(box)

    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()
    #index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()
    index = np.array(cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45)).flatten()
    return boxes_np, confidences_np, index
    
def drawings(image, boxes_np, confidences_np, index):   
    for ind in index:
        x, y, w, h= boxes_np[ind]
        bb_conf = confidences_np[ind]
        conf_text = 'plate: {:.0f}%'.format(bb_conf*100)
        license_text= extract_text(image, boxes_np[ind] )
        
        cv2.rectangle(image,(x, y), (x+w, y+h), (255,0,255),2 )
        cv2.rectangle(image, (x, y-30), (x+w, y), (255,0,255),-1)
        cv2.rectangle(image, (x,y+h), (x+w, y+h+30), (0, 0, 0), -1)
        
        cv2.putText(image, conf_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 1)
        cv2.putText(image, license_text, (x, y+h+27), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 1 )
    return image

def yolo_predictions(img, net):
    input_image, detections = get_detections(img, net)    
    boxes_np, confidences_np, index = non_maximum_suppression(input_image, detections)
    result_img = drawings(input_image, boxes_np, confidences_np, index)
    return result_img

def extract_text(image, bbox):
    x, y, w, h= bbox
    roi = image[y:y+h, x:x+w]
    if 0 in roi.shape:
        return ' '
    else:
        text= pt.image_to_string(roi)
        text= text.strip()
        return text
    

result_image = yolo_predictions(img, net)
cv2.namedWindow('results',cv2.WINDOW_KEEPRATIO)
cv2.imshow('results',result_image)
cv2.waitKey()
cv2.destroyAllWindows()

In [2]:
cap = cv2.VideoCapture('/home/deb/Vídeos/clide.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        print('Impossible to read the frame')
        break
    
    # Assuming yolo_predictions and net are defined elsewhere
    result_image = yolo_predictions(frame, net)
    
    cv2.namedWindow('yolo', cv2.WINDOW_NORMAL)
    cv2.imshow('yolo', result_image)
    
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()


KeyboardInterrupt: 